In [1]:
import networkx as nx
import numpy as np
from collections import defaultdict
import random
from numpy import random
import matplotlib.pyplot as plt

import gurobipy as gp
from gurobipy import GRB
from gurobipy import*
from gurobipy import quicksum

In [37]:
 ### 类缺省属性 class default setup
class Benders_decomposition(object):   
    A = np.array([[0, 0],
                  [0, 0],
                  [0, 0],
                  [0, 0],
                  [-1, -1],
                  [-1, 0],
                  [-1, 0],
                  [0, -1],
                  [0, -1],])

    G = np.array([[1,0,1,0], 
                  [1,0,0,1],
                  [0,1,1,0],
                  [0,1,0,1],
                  [0,0,0,0],
                  [1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1],])
    
    b = np.array([1,1,1,1,-1,0,0,0,0])

    c = np.array([-15.0,-10.0])

    h = np.array([8,9,5,6])
    
    def __init__(self, data):
        self.data = data

In [67]:
 ### 类缺省属性 class default setup
class random_Benders_decomposition(object):   
    size_x = 50
    size_y = 20
    size_constraint  = 10
    A = np.random.randint(-5,5,size=(size_constraint,size_x))

    G = np.random.randint(-20,20,size=(size_constraint,size_y))
    
    b = np.random.randint(-10,10, size=size_constraint)

    c = np.random.randint(-10.0,10.0,size=size_x)

    h = np.random.randint(0.0,10.0,size=size_y)
    
    def __init__(self, data):
        self.data = data

In [68]:
Example = random_Benders_decomposition(0)
size_of_constraints,size_of_x = np.shape(Example.A)
_,size_of_y = np.shape(Example.G)
h = Example.h
c = Example.c
A = Example.A
G = Example.G
b = Example.b
print(A)
print(size_of_x, size_of_y)

[[-1 -4 -1 -5  4 -3  0  1 -4  4 -1  4  1 -2 -2  1 -1 -5 -5  3 -5 -1  0  3
   0  4 -1 -1 -1 -4  1  0  3 -2 -2 -4 -2  4  2 -1  3 -4 -5 -5 -3 -5  2  0
  -1 -2]
 [-3 -1  3 -2 -5  3  4 -4  0  3  4  3  3 -5  0  0 -2 -5 -5  3  1 -2  1 -5
   3  0 -2  0  1 -4  4  4  3  3  3 -5  1  0  1  1  4 -4  0 -2  4  1  4  2
   2  0]
 [ 1 -2  0 -2  0 -5 -1  4  4 -2 -2 -3  3 -1  0 -1  4  4  2  2  1  2 -4 -1
  -5 -3 -2  4  4  4  4  0 -4  0 -2  2  2 -3  4  2  2  2 -5 -1 -3  2 -4  2
   3  2]
 [ 4  2 -3  0  0  4  3  2 -3 -2 -5  1 -2 -1 -3  3  2 -4 -4  1  2 -2  4 -2
   1 -2 -1 -5  1  2 -5  4  2  2 -3 -1  2  3 -4 -5 -1  2 -2  3 -1 -3 -3  3
  -1  1]
 [-2  4 -1  2 -5  0 -3 -1 -4  4  1  3  3  1 -5 -2 -2  0 -4  0  0 -4  4 -5
  -2 -4 -3 -5  3  2  4 -5  3 -4 -3  3 -1 -3 -1  3  1 -1 -3  4 -3 -3  1 -3
   1  3]
 [-4  2  0  0  4 -3  2  2  0  3 -3  0 -5 -5  1  0  1  2  1  1  3  1 -3  2
  -2 -5  0  0  4  2  0 -3 -2 -1  3 -1  1 -5  0  3  4 -1  2 -1  2 -3  3 -1
   2 -2]
 [ 1 -4  0  1 -1  2 -5  3  1  3 -5 -3 -4 -3  1  4  4 -1 -1

In [69]:
m = gp.Model("Benders Decomp Gurobi")
x = m.addVars(size_of_x, vtype=GRB.BINARY, name="x")
y = m.addVars(size_of_y, vtype=GRB.CONTINUOUS, lb = 0, ub = 5, name="y")

In [70]:
# Set objective
m.setObjective(quicksum(h[i] * y[i] for i in range(size_of_y)) + 
              quicksum(c[i] * x[i] for i in range(size_of_x)), GRB.MAXIMIZE)

In [71]:
# Add constraints:
for item in range(size_of_constraints):
        
    m.addConstr(quicksum(A[item, i] * x[i]  for i in range(size_of_x)) + 
                quicksum(G[item, j] * y[j]  for j in range(size_of_y)) <= b[item], name=f'constraint_{item}')

In [72]:
m.optimize()
#print (m.display())

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads
Optimize a model with 10 rows, 70 columns and 649 nonzeros
Model fingerprint: 0x25853d6a
Variable types: 20 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 5e+00]
  RHS range        [2e+00, 9e+00]
Found heuristic solution: objective 40.5748168
Presolve time: 0.00s
Presolved: 10 rows, 70 columns, 649 nonzeros
Variable types: 20 continuous, 50 integer (50 binary)
Found heuristic solution: objective 127.3023543

Root relaxation: objective 3.360877e+02, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     336.0877309  336.08773  0.00%     -    0s

Explored 1 nodes (10 simplex ite

In [73]:
print("Gurobi runtime is", m.Runtime ,"seconds")

Gurobi runtime is 0.00901031494140625 seconds


In [45]:
x_list = [str.replace(str.replace(v.varName,"]",""),"x[","") for v in m.getVars() if v.x == 1 and v.varName[0] =='x']
        
print(x_list)

['9', '11', '15', '19', '20', '21', '22', '24', '29', '32', '33', '35', '37', '39', '40', '44']


In [46]:
m.ObjVal

260.8810305909356

In [32]:
try:
    _,size_of_x = np.
    
    # Create a new model
    m = gp.Model("Benders Decomp Gurobi")

    # Create variables
    x = m.addVars(edge_number, Q, vtype=GRB.BINARY, name="x")
    
    print(x[0,1])
    # Set objective
    m.setObjective(quicksum(quicksum((p*j+1) * c_e[i,j]  * x[i,j] for i in range(edge_number)) 
                            for j in range(Q)), GRB.MAXIMIZE)

    # Add constraint 1:
    for vertex in range(1, Q + 1):
    
        index = constraint_1(network, Q, edge_dictionary, compelete_label, vertex)

        # 
        m.addConstr(quicksum( quicksum(x[i,j] for j in range(Q)) for i in index) == 1, name=f'constraint_1_vertex_{vertex}')

    # Add constraint 2:  
    for vertex in range(1, Q + 1):
    
        index_out, index_in = constraint_2(network, Q, edge_dictionary, compelete_label, vertex)
    
        #print(index_out, index_in, level_coeff[index_in])
    
        m.addConstr( quicksum(quicksum( (j+1) * x[i,j] for j in range(Q)) for i in index_out) - quicksum( quicksum( (n+1) * x[m,n] for n in range(Q)) for m in index_in)  == 1, name=f'constraint_2_vertex_{vertex}')

    # Add constraint 3:
    m_t_u,m_t_l = capacity_subtree_limit_generator(Q)
    print(m_t_u,m_t_l)
    
    for level in range(Q):
        m.addConstr ( quicksum(x[i,int(level)] for i in range(Q)) >= m_t_l[int(level)], name=f'constraint_3_Buslowerlevel_{level}')
        m.addConstr ( quicksum(x[i,int(level)] for i in range(Q)) <= m_t_u[int(level)], name=f'constraint_3_Busupperlevel_{level}')
    
    
    # Optimize model
    m.optimize()
    print (m.display())

    #$for v in m.getVars():
        #print('%s %g' % (v.varName, v.x))

    x_list = [str.replace(str.replace(v.varName,"]",""),"x[","") for v in m.getVars() if v.x == 1 and v.varName[0] =='x']
        
    print(x_list )
    
    #print('Obj: %g' % m.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

SyntaxError: invalid syntax (<ipython-input-32-a5bdadecf18b>, line 2)